# Импорты

In [12]:
import numpy as np 
import pandas as pd

import os
import shutil

## Откроем датасеты разметки и посмотрим содержимое

In [2]:
df_train = pd.read_csv("/kaggle/input/detection-rtuailab-task2/training_demo/annotations/train_labels.csv")

In [7]:
df_train.head()

,filename,width,height,class,xmin,ymin,xmax,ymax
0,all11.jpg,800,600,MCC_SD card connector,45,385,114,462
1,all11.jpg,800,600,Piezo Buzzer,54,238,127,287
2,all11.jpg,800,600,LCD display place,156,118,305,171
3,all11.jpg,800,600,LEDs,142,185,342,284
4,all11.jpg,800,600,Push Buttons,125,330,281,496


In [8]:
df_test = pd.read_csv("/kaggle/input/detection-rtuailab-task2/training_demo/annotations/test_labels.csv")
df_test.head()

,filename,width,height,class,xmin,ymin,xmax,ymax
0,all.jpg,800,600,MCC_SD card connector,44,19,96,51
1,all.jpg,800,600,Piezo Buzzer,171,27,205,53
2,all.jpg,800,600,LCD display place,251,68,288,135
3,all.jpg,800,600,LEDs,173,64,242,150
4,all.jpg,800,600,microBoard socket,195,158,353,210


## Балансировка классов оставляет желать лучшего)

In [6]:
df_train["class"].value_counts()

class
GLCD Display Place            57
LEDs                          39
LCD display place             38
Push Buttons                  38
microBoard socket             38
Jumpers                       38
Platamicroboard               35
DebugUSB                      35
MCC_SD card connector         34
DisplayEasyTFT                34
ButtonPressLevel              34
Processor                     34
Ports                         33
Piezo Buzzer                  32
Joystick                      32
Buzzer                        28
PlataGreenWithProcessor       23
Digit                         22
ProcessorPlace                21
PortsBlueOrange               20
PlataOrange                   11
DebugBlue                     11
DebugOrange                   11
DisplayBlue                   11
click                         11
PlataGreenWithoutProcessor     9
PlataBlue                      9
Name: count, dtype: int64

In [9]:
df_test["class"].value_counts()

class
GLCD Display Place            24
Processor                     18
Ports                         18
DisplayEasyTFT                17
Buzzer                        17
Joystick                      17
ButtonPressLevel              17
DebugUSB                      17
Digit                         15
LCD display place             13
LEDs                          13
microBoard socket             13
Jumpers                       13
Push Buttons                  13
PortsBlueOrange               12
MCC_SD card connector         12
PlataGreenWithProcessor       11
ProcessorPlace                10
Piezo Buzzer                  10
Platamicroboard                9
DisplayBlue                    9
DebugBlue                      8
DebugOrange                    7
PlataOrange                    7
PlataBlue                      6
click                          5
PlataGreenWithoutProcessor     4
Name: count, dtype: int64

## Преобразуем данные в формат для YoloV11

In [10]:
# Папки с исходными данными
dataset_path = "/kaggle/input/detection-rtuailab-task2/training_demo"
output_path = "/kaggle/working/"

In [13]:
# Создаём необходимые директории
for split in ["train", "test"]:
    os.makedirs(os.path.join(output_path, split, "images"), exist_ok=True)
    os.makedirs(os.path.join(output_path, split, "labels"), exist_ok=True)

In [16]:
# Создаём словарь для кодирования классов
class_labels = train_df["class"].unique().tolist()
class_map = {cls: i for i, cls in enumerate(class_labels)}

In [17]:
def convert_and_save(df, split):
    for filename in df["filename"].unique():
        img_path = os.path.join(dataset_path, "images", split, filename)
        label_path = os.path.join(output_path, split, "labels", filename.replace(".jpg", ".txt"))
        
        # Копируем изображение
        if os.path.exists(img_path):
            shutil.copy(img_path, os.path.join(output_path, split, "images", filename))
        
        # Фильтруем строки, относящиеся к этому изображению
        img_data = df[df["filename"] == filename]
        img_width = img_data["width"].values[0]
        img_height = img_data["height"].values[0]

        with open(label_path, "w") as f:
            for _, row in img_data.iterrows():
                class_id = class_map[row["class"]]
                x_center = ((row["xmin"] + row["xmax"]) / 2) / img_width
                y_center = ((row["ymin"] + row["ymax"]) / 2) / img_height
                bbox_width = (row["xmax"] - row["xmin"]) / img_width
                bbox_height = (row["ymax"] - row["ymin"]) / img_height
                
                f.write(f"{class_id} {x_center:.6f} {y_center:.6f} {bbox_width:.6f} {bbox_height:.6f}\n")

In [18]:
# Конвертируем и сохраняем аннотации
convert_and_save(train_df, "train")
convert_and_save(test_df, "test")

In [19]:
# Сохраняем файл классов
with open(os.path.join(output_path, "classes.txt"), "w") as f:
    for cls in class_labels:
        f.write(f"{cls}\n")

print("status: ok")

status: ok


In [21]:
shutil.make_archive('/kaggle/working/train_zip', 'zip', '/kaggle/working', base_dir='train')
shutil.make_archive('/kaggle/working/test_zip', 'zip', '/kaggle/working', base_dir='test')

'/kaggle/working/test_zip.zip'

### Вообще очень интересные картинки получаются я смотрю они все разного размера и вообще иногда даже человеку непонятно че там вообще